# 1. Загрузка файла

Для выполнения задания я скачала с помощью youtube_dl серию "Смысл жизни" Смешариков -- она очень подходит под настроение. Устанавливала библиотеку с мастер-ветки, так как некоторые ошибки, появляющиеся при загрузке, исправлены только в ней.

In [1]:
! pip install git+https://github.com/ytdl-org/youtube-dl.git@master#egg=youtube_dl

  Cloning https://github.com/ytdl-org/youtube-dl.git (to revision master) to /tmp/pip-install-z858p26d/youtube-dl_4d81c373692941d9b37c544f8fa6ed01
  Running command git clone --filter=blob:none --quiet https://github.com/ytdl-org/youtube-dl.git /tmp/pip-install-z858p26d/youtube-dl_4d81c373692941d9b37c544f8fa6ed01
  Resolved https://github.com/ytdl-org/youtube-dl.git to commit be008e657d79832642e2158557c899249c9e31cd
  Preparing metadata (setup.py) ... done
  Created wheel for youtube_dl: filename=youtube_dl-2021.12.17-py2.py3-none-any.whl size=1944011 sha256=5cc06e9df60ca3fdde4b48a9abe48a5dfb183aede5d8c9b95acb08a9c479c403
  Stored in directory: /tmp/pip-ephem-wheel-cache-pnks6bxo/wheels/b8/03/62/9c414b89a26da510b0a6d984b0ba74200d591e3d0abfa72aa8
Successfully built youtube_dl


In [2]:
import youtube_dl
ydl_opts = {
'format': 'bestaudio',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
'writesubtitles': True,
'writeautomaticsub': True,
'subtitlesformat':'best',
'subtitleslangs': ['ru']
}
url = 'https://www.youtube.com/watch?v=Pjxr08Cw59w'
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
  ydl.download([url])
print("Download Successful!")

[youtube] Pjxr08Cw59w: Downloading webpage
[youtube] Pjxr08Cw59w: Downloading player d23221b6
[info] Writing video subtitles to: Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.ru.vtt
[dashsegments] Total fragments: 1
[download] Destination: Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.webm
[download] 100% of 6.31MiB in 00:00
[ffmpeg] Destination: Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.wav
Deleting original file Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.webm (pass -k to keep)
Download Successful!


Обрезаю видео:

In [3]:
! pip install pydub

In [4]:
from pydub import AudioSegment

audio_file = AudioSegment.from_file('Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.wav')
start_time = 20000
end_time = 131000

trimmed_audio = audio_file[start_time:end_time]
trimmed_audio = trimmed_audio.set_frame_rate(16000)
trimmed_audio.export("audio.wav", format="wav")

<_io.BufferedRandom name='audio.wav'>

Чищу файл с субтитрами и убираю дублирующиеся строки:

In [1]:
import re
bad_words = ['-->','</c>']


with open('Смысл жизни - Смешарики 2D _ Мультфильмы для детей-Pjxr08Cw59w.ru.vtt', encoding='utf-8') as oldfile, open('youtube_sb.txt', 'w') as newfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            newfile.write(line)


seen = []
with open('youtube_sb.txt') as result:
    lines = result.readlines()
    for line in lines:
      if line not in seen:
        seen.append(line)

In [2]:
seen = [each.strip() for each in seen]
youtube_sb = seen[7:30] + [seen[30][:9]]
youtube_sb = ' '.join(youtube_sb)

# 2. ASR

In [3]:
evaluations = {}

In [4]:
evaluations['youtube'] = youtube_sb

In [5]:
audio_path = 'audio.wav'

## 2.1 SpeechRecognition

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install git+https://github.com/openai/whisper.git soundfile

In [6]:
import speech_recognition as sr

In [7]:
r = sr.Recognizer()
with sr.AudioFile(audio_path) as source:
    audio = r.record(source)

In [8]:
whisper_sb = r.recognize_whisper(audio)

In [9]:
evaluations['whisper'] = whisper_sb

In [10]:
google_sb = r.recognize_google(audio, language='ru')

Почему-то весь текст распознать не удалось

In [11]:
google_sb

'Иногда случается что вещи теряют свой смысл Вот смысл окна например чтобы в него смотреть а порой смотришь в него смотреть там нечего ничего интересного или вот какой смысл умываться по утрам чтобы быть бодрым А какой смысл быть по утрам бодрым какой вообще смысл быть по утрам непонятно и непонятно то ли жизнь всегда была бессмысленной а ты просто не замечал этого то ли смысл был но куда-то быть'

In [12]:
evaluations['google'] = google_sb

# 2.2 MMS

При запуске стандартного API MMS у меня возникали ошибки, поэтому я воспользовалась другим пакетом для работы с MMS: https://github.com/abdeladim-s/easymms

Но ошибки все еще возникают, так как пакет работает с CUDA 11, а по дефолту в колабе стоит CUDA 12, поэтому после выполнения ячеек с импортами и установками нужно перезапустить среду

In [ ]:
import os

!apt install ffmpeg
!apt install sox
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install -U --pre torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118
!pip install git+https://github.com/abdeladim-s/easymms
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [20]:
# @title Download Model { display-mode: "form" }

model = 'mms1b_fl102' #@param ["mms1b_fl102", "mms1b_l1107", "mms1b_all"] {allow-input: true}

if model == "mms1b_fl102":
  !wget -P ./models 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'

elif model == "mms1b_l1107":
  !wget -P ./models 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

elif model == "mms1b_all":
  !wget -P ./models 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-12-18 23:16:38--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.108, 3.163.189.96, 3.163.189.14, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4851043301 (4.5G) [binary/octet-stream]
Saving to: ‘./models/mms1b_fl102.pt’

mms1b_fl102.pt      100%[===================>]   4.52G   206MB/s    in 29s     

2023-12-18 23:17:07 (158 MB/s) - ‘./models/mms1b_fl102.pt’ saved [4851043301/4851043301]



In [13]:
files = [f'/content/{audio_path}']

In [14]:
from easymms.models.asr import ASRModel

In [ ]:
asr = ASRModel(model='/content/models/mms1b_fl102.pt')

transcriptions = asr.transcribe(files, lang='rus', align=False)

In [16]:
mms_sb = transcriptions[0]

In [17]:
evaluations['mms'] = mms_sb

Чекпойнт, потому что на этом месте обычно заканчивается ОЗУ в колабе

In [18]:
import json

In [23]:
with open("evaluations.json", "w", encoding='utf-8') as outfile:
    json.dump(evaluations, outfile, ensure_ascii=False, indent='\t')

# 2.3 Transformers

In [ ]:
!pip install huggingsound

Модель, которую я использовала: https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-russian

In [2]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian")


INFO:huggingsound.speech_recognition.model:Loading model...


In [4]:
transcriptions = model.transcribe(['audio.wav'])

100%|██████████| 1/1 [05:10<00:00, 310.13s/it]


In [7]:
import json

In [8]:
with open('evaluations.json', 'r', encoding='utf-8') as f:
    evaluations = json.load(f)

In [10]:
evaluations['wav2vec'] = transcriptions[0]['transcription']

In [11]:
with open("evaluations.json", "w", encoding='utf-8') as outfile:
    json.dump(evaluations, outfile, ensure_ascii=False, indent='\t')

# 3. Оценка

In [ ]:
!pip install evaluate jiwer

In [12]:
gold_sb = '''Иногда случается, что вещи теряют свой смысл. Вот смысл окна, например, чтобы в него смотреть. А порой смотришь в него, а смотреть там нечего. Ничего интересного. Или вот, какой смысл умываться по утрам? Чтобы быть бодрым? А какой смысл быть по утрам бодрым? Какой вообще смысл быть по утрам? Непонятно. И непонятно, то ли жизнь всегда была бессмысленной, а ты просто не замечал этого, то ли смысл был, но куда-то затерялся. Не лежи на снегу, простудишься. Ну и что. А смысл не простуживаться? Чтобы не болеть. А смысл не болеть?  Чтобы быть здоровым. А смысл быть здоровым? Чудак! Тебе что, не нравится быть здоровым? Причем здесь нравится или не нравится. Я не хочу быть бессмысленно здоровым. Ай, тебе нужен смысл? Да. Во всем во всем? Неплохо бы. Тогда тебе нужна кузинатра. Какая еще кузинатра? Кузинатра, дающая смысл. Да, она мне нужна. Но где ее искать? Это очень далеко. За смыслом хоть на край света. Веди меня.'''

In [15]:
from evaluate import load
wer = load("wer")

Лучший результат показала Whisper, остальные справились значительно хуже. Основная проблема, как мне кажется, в орфографическом отображении звучащего текста -- фонетически полученный транслит верен, но записан он неправильно.

In [24]:
for key, value in evaluations.items():
  wer_score = wer.compute(predictions=[value], references=[gold_sb]) #wer от huggingface не учитывает знаки препинания, поэтому не убирала их
  print(f"Система ASR: {key}", f"WER: {wer_score}", "Распознанный текст:", sep='\n')
  print(value)
  print('\n')

Система ASR: youtube
WER: 0.6025641025641025
Распознанный текст:
иногда случается что вещи теряют свой смысл вот смысл окна например чтобы в него смотреть а порой смотришь в него или вот какой смысл умываться по утрам чтобы быть бодрым а какой смысл быть по утрам бодром какой вообще смысл быть по утра не понятно и не понятно то ли жизнь всегда была бессмысленный а ты просто не замечал этого не лежи на снегу простудишься но изд-во а смысл не прослушивается чтобы не болеть а смысл нибали чтобы быть здоровым ёлки-иголки а смысл быть здоровым чудак тебе что не нравится быть здоровым причем здесь нравится или не нравится я не хочу быть бессмысленно здоровым тебе нужен смысл да во всем во всем неплохо бы тогда тебе нужно кузина то кузина продающие смыслу да она мне нужно многое риска это очень за смыслом на кости идеи меня


Система ASR: whisper
WER: 0.20512820512820512
Распознанный текст:
 Иногда случается, что вещи теряют свой смысл. Вот смысл окна, например, чтобы в него смотреть, а порой